In [2]:
471859200 / 1024**2 # min size in MB

450.0

In [3]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

23/10/25 15:27:12 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [7]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [9]:
df = spark.read.format("csv").load("hdfs://nn:9000/sf.csv")

In [10]:
%%time
df.count()

CPU times: user 34.7 ms, sys: 6.81 ms, total: 41.5 ms
Wall time: 20.6 s


6016057

In [12]:
df.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c25,_c26,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34
0,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
1,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
2,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26
3,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,...,Alarm,1,TRUCK,2,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20
4,212933533,B02,21127914,Alarms,10/20/2021,10/20/2021,10/20/2021 10:08:47 PM,10/20/2021 10:09:53 PM,10/20/2021 10:10:07 PM,10/20/2021 10:11:55 PM,...,Alarm,1,CHIEF,3,3,6,Tenderloin,212933533-B02,POINT (-122.41243514072728 37.78347684038771),36


In [13]:
df = (spark.read.format("csv")
      .option("header", True)
      .load("hdfs://nn:9000/sf.csv"))

In [14]:
df.limit(5).toPandas()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,...,Alarm,1,TRUCK,2,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20
3,212933533,B02,21127914,Alarms,10/20/2021,10/20/2021,10/20/2021 10:08:47 PM,10/20/2021 10:09:53 PM,10/20/2021 10:10:07 PM,10/20/2021 10:11:55 PM,...,Alarm,1,CHIEF,3,3,6,Tenderloin,212933533-B02,POINT (-122.41243514072728 37.78347684038771),36
4,221202543,E41,22054815,Alarms,04/30/2022,04/30/2022,04/30/2022 06:35:58 PM,04/30/2022 06:37:28 PM,04/30/2022 06:37:43 PM,04/30/2022 06:38:17 PM,...,Alarm,1,ENGINE,4,4,2,Russian Hill,221202543-E41,POINT (-122.4233369425531 37.799534868680034),32


In [15]:
df.dtypes

[('Call Number', 'string'),
 ('Unit ID', 'string'),
 ('Incident Number', 'string'),
 ('Call Type', 'string'),
 ('Call Date', 'string'),
 ('Watch Date', 'string'),
 ('Received DtTm', 'string'),
 ('Entry DtTm', 'string'),
 ('Dispatch DtTm', 'string'),
 ('Response DtTm', 'string'),
 ('On Scene DtTm', 'string'),
 ('Transport DtTm', 'string'),
 ('Hospital DtTm', 'string'),
 ('Call Final Disposition', 'string'),
 ('Available DtTm', 'string'),
 ('Address', 'string'),
 ('City', 'string'),
 ('Zipcode of Incident', 'string'),
 ('Battalion', 'string'),
 ('Station Area', 'string'),
 ('Box', 'string'),
 ('Original Priority', 'string'),
 ('Priority', 'string'),
 ('Final Priority', 'string'),
 ('ALS Unit', 'string'),
 ('Call Type Group', 'string'),
 ('Number of Alarms', 'string'),
 ('Unit Type', 'string'),
 ('Unit sequence in call dispatch', 'string'),
 ('Fire Prevention District', 'string'),
 ('Supervisor District', 'string'),
 ('Neighborhooods - Analysis Boundaries', 'string'),
 ('RowID', 'string')

In [16]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [17]:
df

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

In [18]:
# Learn strategies for transforming the data

In [20]:
from pyspark.sql.functions import col, expr

In [21]:
col("x")

Column<'x'>

In [23]:
expr("x + 1")

Column<'(x + 1)'>

In [24]:
expr("x + 1").alias("plusone")

Column<'(x + 1) AS plusone'>

In [31]:
df.select(expr("`Call Date`")).limit(3).toPandas()

,Call Date
0,05/01/2022
1,01/19/2022
2,05/03/2021


In [32]:
df.select(expr("to_date(`Call Date`, 'MM/dd/yyyy')")).limit(3).toPandas()

,"to_date(Call Date, MM/dd/yyyy)"
0,2022-05-01
1,2022-01-19
2,2021-05-03


In [36]:
df.select(expr("to_date(`Call Date`, 'MM/dd/yyyy')").alias("CallDate"))

DataFrame[CallDate: date]

In [35]:
df.select(expr("to_date(`Call Date`, 'MM/dd/yyyy')").alias("CallDate")).limit(3).toPandas()

,CallDate
0,2022-05-01
1,2022-01-19
2,2021-05-03


In [19]:
# GOAL: write this DataFrame to Parquet, with no spaces in the columns

In [43]:
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
cols[:5]

[Column<'Call Number AS Call_Number'>,
 Column<'Unit ID AS Unit_ID'>,
 Column<'Incident Number AS Incident_Number'>,
 Column<'Call Type AS Call_Type'>,
 Column<'Call Date AS Call_Date'>]

In [47]:
df.select(cols).write.format("parquet").save("hdfs://nn:9000/sf.parquet")

In [49]:
!hdfs dfs -ls hdfs://nn:9000/sf.parquet/

Found 18 items
-rw-r--r--   3 root supergroup          0 2023-10-25 15:46 hdfs://nn:9000/sf.parquet/_SUCCESS
-rw-r--r--   3 root supergroup   27805433 2023-10-25 15:44 hdfs://nn:9000/sf.parquet/part-00000-0da1dc41-2d26-4bc7-a89c-21b7bc2973e2-c000.snappy.parquet
-rw-r--r--   3 root supergroup   27788944 2023-10-25 15:44 hdfs://nn:9000/sf.parquet/part-00001-0da1dc41-2d26-4bc7-a89c-21b7bc2973e2-c000.snappy.parquet
-rw-r--r--   3 root supergroup   40477903 2023-10-25 15:44 hdfs://nn:9000/sf.parquet/part-00002-0da1dc41-2d26-4bc7-a89c-21b7bc2973e2-c000.snappy.parquet
-rw-r--r--   3 root supergroup   36016086 2023-10-25 15:44 hdfs://nn:9000/sf.parquet/part-00003-0da1dc41-2d26-4bc7-a89c-21b7bc2973e2-c000.snappy.parquet
-rw-r--r--   3 root supergroup   36031567 2023-10-25 15:45 hdfs://nn:9000/sf.parquet/part-00004-0da1dc41-2d26-4bc7-a89c-21b7bc2973e2-c000.snappy.parquet
-rw-r--r--   3 root supergroup   36080343 2023-10-25 15:45 hdfs://nn:9000/sf.parquet/part-00005-0da1dc41-2d26-4bc7-a89c-21b7bc